## ExMAS
> Equilibrium matching



Here I:

* generatre demand
* compute groups with ExMAS
* compute costs for groups and travellers
* apply Subgroup splitting protocol
* run ExMAS
* prune groups with Hermetic
* see if there is a group in ExMAS which is then non HERMETIC 


In [1]:
import os
import pandas as pd
import math
import seaborn as sns
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
cwd = os.getcwd()
%load_ext autoreload
%autoreload 2

In [2]:
os.chdir(os.path.join(cwd,'../../..'))
import ExMAS.main
import ExMAS.utils
from ExMAS.utils import inData as inData
from ExMAS.main import matching
from ExMAS.extras import games, pricings, prunings, pipeline

In [3]:
params = ExMAS.utils.get_config('ExMAS/spinoffs/game/pipe.json') # load the default 
params.t0 = pd.to_datetime(params.t0)
params.matching_obj = 'u_pax'
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph
params.nP = 100
params.simTime = 0.1
params.shared_discount = 0.3
inData = ExMAS.utils.generate_demand(inData, params)  # generate requests

In [4]:

params.time_cost = params.VoT # travellers' cost per travel time
params.wait_cost = params.time_cost*1.5 # and waiting
params.sharing_penalty_fixed = 0 # fixed penalty (EUR) per 
params.sharing_penalty_multiplier = 0 # fixed penalty (EUR) per 

params.veh_cost = 1.3*params.VoT/params.avg_speed # operating costs per kilometer
params.fixed_ride_cost = 0.5 # ride fixed costs (per vehicle)

In [5]:
# def test_me(inData):
#     inData = ExMAS.utils.generate_demand(inData, params)  # generate requests
#     inData = ExMAS.main(inData, params, plot = False)
#     inData = games.prepare_PoA(inData)
#     inData = pricings.update_costs(inData, params)
#     PRICINGS = dict()  # pricings to apply and their names
#     PRICINGS['SUBGROUP'] = pricings.subgroup_split
#     for PRICING, pricing in PRICINGS.items():
#         inData = pricing(inData)  # apply pricing strategy
#     inData = prunings.algo_EXMAS(inData, price_column='SUBGROUP')  # apply pruning strategies for a given pricing strategy
#     inData = pipeline.single_eval(inData, params,
#                      MATCHING_OBJS = ['total_group_cost'],  # this can be more
#                      PRUNINGS = ['EXMAS'],  # and this can be more
#                      PRICING = 'SUBGROUP',  # this is taken from first level loop
#                      minmax = ['min'], EXPERIMENT_NAME = 'debug', store_res = False)  # direction BPoA, WPoAplot_im(inData)
#     inData = prunings.algo_HERMETIC(inData, price_column='SUBGROUP')  # apply pruning strategies for a given pricing strategy
#     rides = inData.sblts.rides
#     wrong_ones = rides[rides.selected & ~rides.pruned_HERMETIC]
#     print(wrong_ones.shape)
#     if wrong_ones.shape[0]>0:
#         dwwd
# for i in range(10):
#     print(i)
#     test_me(inData)

In [6]:
inData = ExMAS.main(inData, params, plot = False)
KPIs = inData.sblts.res.to_frame('u_pax')

In [7]:
inData = games.prepare_PoA(inData)

20-04-21 09:19:35-WARNING-Prepare for game:  1/6
20-04-21 09:19:36-WARNING-Prepare for game:  2/6
20-04-21 09:19:37-WARNING-Prepare for game:  3/6
20-04-21 09:19:37-WARNING-Prepare for game:  4/6
20-04-21 09:20:25-WARNING-Prepare for game:  5/6
20-04-21 09:20:28-WARNING-Prepare for game:  6/6


In [8]:
inData = pricings.update_costs(inData, params)

In [9]:
PRICINGS = dict()  # pricings to apply and their names
PRICINGS['SUBGROUP'] = pricings.subgroup_split
for PRICING, pricing in PRICINGS.items():
    inData = pricing(inData)  # apply pricing strategy

In [10]:
inData = prunings.algo_EXMAS(inData, price_column='SUBGROUP')  # apply pruning strategies for a given pricing strategy

In [11]:
inData = pipeline.single_eval(inData, params,
                     MATCHING_OBJS = ['total_group_cost'],  # this can be more
                     PRUNINGS = ['EXMAS'],  # and this can be more
                     PRICING = 'SUBGROUP',  # this is taken from first level loop
                     minmax = ['min'], EXPERIMENT_NAME = 'debug', store_res = False)  # direction BPoA, WPoAplot_im(inData)

20-04-21 09:20:56-WARNING-Prunings:  ['EXMAS']
20-04-21 09:20:56-WARNING-Pruned nRides 1173/1173
20-04-21 09:20:56-WARNING-Mutually exclusives 0
20-04-21 09:20:56-WARNING-Experiment-debug_Pricing-SUBGROUP_Objective-['total_group_cost']_Pruning-['EXMAS']_minmax-min


### HERMETIC 


only Hermetic groups are left in the search space

Subgroup H of Group G is not hermetic if somoeone wants to leave H for G

In [12]:
inData = prunings.algo_HERMETIC(inData, price_column='SUBGROUP')  # apply pruning strategies for a given pricing strategy

wrong ones (not Hermetic and in the solution)

In [13]:
rides = inData.sblts.rides
rides[rides.selected & ~rides.pruned_HERMETIC].indexes

Series([], Name: indexes, dtype: object)

In [ ]:
RIDEID = 100
RIDE = rides.loc[RIDEID]

In [ ]:
RIDE.to_frame().T.subgroups

In [ ]:
subgroups = RIDE.subgroups
subgroup_indexes = rides.loc[subgroups][['indexes_set']]  # travellers indexes in the subgroups
subgroup_indexes
rides.loc[subgroups].cost_efficiency

In [ ]:
del inData.sblts.rides_multi_index['SUBGROUP']

In [50]:
def get_subgroup_price(r):
        # assigns traveller prices by their best alternatives
        C_G = r.total_group_cost
        reference_cost_eff = r.cost_efficiency
        indexes_set = r.indexes_set  # travellers of this group
        subgroups = r.subgroups  # subgroups of this group
        subgroup_indexes = rides.loc[subgroups][['indexes_set']]  # travellers indexes in the subgroups

        z_i = dict()  # return dict to populate - prices
        fi_i = dict() # return dict to populate - best subgroups
        while len(indexes_set) > 0:  # until everyone is assigned
            effs = rides.loc[subgroups].cost_efficiency  # see the efficiencies of remaining subgroups
            J, z = effs.idxmin(), effs.min()  # pick up the subgroup of greatest efficiency and its index

            for i in rides.loc[J].indexes_set:
                z_i[i] = z  # assign the prices
                fi_i[i] = J # memorize the group

            indexes_set = indexes_set - rides.loc[J].indexes_set  # remove those from the best group
            subgroup_indexes = rides.loc[subgroups][['indexes_set']]
            subgroup_indexes['f'] = subgroup_indexes.apply(
                lambda x: len(rides.loc[J].indexes_set.intersection(x.indexes_set)) == 0, axis=1)  # update which
            # subgroups remaining assignable
            subgroups = subgroup_indexes[subgroup_indexes.f].index  # filter to those not assigned
            # loop and assign the ones who are not assigned left

        # cost splitting
        z_i = pd.Series(z_i, name='z') 
        fi_i = pd.Series(fi_i, name='fi')

        e = C_G - sum(z_i) # excess

        # check budget balance

        if e >= -0.0000000001:  # if positive excees, distribute the rest equally to each one
            c_i = z_i + e / r.degree            

        else:  # otherwise let's see other subgroups
            # Sort the sets fi in increasing average costs
            df = pd.concat([fi_i, z_i], axis=1).sort_values('z')
            df.index.name = 'i'            
            df['c_i'] = df.z.copy()  # set the costs to the ones generated with the algo above

            J_ref = reference_cost_eff  # set the cut-off for reference (will be increased in the loop)
            
            while True:  # loop
                J, z = df.loc[df[df.z > J_ref].z.idxmin()].fi, df[df.z > J_ref].z.min() # find the first group beyong the threshold
                
                df['c_i'] = df.apply(lambda row: reference_cost_eff if row.z == z else row.c_i, axis = 1)  # update costs of this group to reference ones                

                if df.c_i.sum() > C_G: # see if now excess is OK
                    
                    J_ref = df.loc[rides.loc[J].indexes_set].z.min()  # move to the next index                    

                else:
                    print(df)
                    print(J, z)
                    # distribute remainings
                    c = (C_G - df[~(df.fi == J)].c_i.sum()) / rides.loc[J].degree
                    df.loc[df.z == z, 'c_i'] = c
                    break
            c_i = df.c_i



        assert abs(C_G - c_i.sum()) < 0.0001

        return c_i
rides = inData.sblts.rides
df = get_subgroup_price(rides.loc[612])

     fi         z       c_i
i                          
93   93  2.464200  2.464200
65  194  4.088275  3.537225
29  194  4.088275  3.537225
194.0 4.088274999999999
10.611675 10.611675


In [46]:
df.z.idxmin()

65

In [52]:

def subgroup_split(inData):

    rm = inData.sblts.rides_multi_index
    rides = inData.sblts.rides

    prices = list()

    def get_subgroup_price(r):
        # assigns traveller prices by their best alternatives
        C_G = r.total_group_cost
        reference_cost_eff = r.cost_efficiency
        indexes_set = r.indexes_set  # travellers of this group
        subgroups = r.subgroups  # subgroups of this group
        subgroup_indexes = rides.loc[subgroups][['indexes_set']]  # travellers indexes in the subgroups

        z_i = dict()  # return dict to populate - prices
        fi_i = dict() # return dict to populate - best subgroups
        while len(indexes_set) > 0:  # until everyone is assigned
            effs = rides.loc[subgroups].cost_efficiency  # see the efficiencies of remaining subgroups
            J, z = effs.idxmin(), effs.min()  # pick up the subgroup of greatest efficiency and its index

            for i in rides.loc[J].indexes_set:
                z_i[i] = z  # assign the prices
                fi_i[i] = J # memorize the group

            indexes_set = indexes_set - rides.loc[J].indexes_set  # remove those from the best group
            subgroup_indexes = rides.loc[subgroups][['indexes_set']]
            subgroup_indexes['f'] = subgroup_indexes.apply(
                lambda x: len(rides.loc[J].indexes_set.intersection(x.indexes_set)) == 0, axis=1)  # update which
            # subgroups remaining assignable
            subgroups = subgroup_indexes[subgroup_indexes.f].index  # filter to those not assigned
            # loop and assign the ones who are not assigned left

        # cost splitting
        z_i = pd.Series(z_i, name='z') 
        fi_i = pd.Series(fi_i, name='fi')

        e = C_G - sum(z_i) # excess

        # check budget balance

        if e >= -0.0000000001:  # if positive excees, distribute the rest equally to each one
            c_i = z_i + e / r.degree            

        else:  # otherwise let's see other subgroups
            # Sort the sets fi in increasing average costs
            df = pd.concat([fi_i, z_i], axis=1).sort_values('z')
            df.index.name = 'i'            
            df['c_i'] = df.z.copy()  # set the costs to the ones generated with the algo above

            J_ref = reference_cost_eff  # set the cut-off for reference (will be increased in the loop)
            
            while True:  # loop
                J, z = df.loc[df[df.z > J_ref].z.idxmin()].fi, df[df.z > J_ref].z.min() # find the first group beyong the threshold
                
                df['c_i'] = df.apply(lambda row: reference_cost_eff if row.z == z else row.c_i, axis = 1)  # update costs of this group to reference ones                

                if df.c_i.sum() > C_G: # see if now excess is OK
                    
                    J_ref = df.loc[rides.loc[J].indexes_set].z.min()  # move to the next index                    

                else:
                    # distribute remainings
                    c = (C_G - df[~(df.fi == J)].c_i.sum()) / rides.loc[J].degree
                    df.loc[df.z == z, 'c_i'] = c
                    break
            c_i = df.c_i



        assert abs(C_G - c_i.sum()) < 0.0001

        return c_i

    for i, r in rides.iterrows():
        ret = get_subgroup_price(r)
        for j, row in ret.iteritems():
            prices.append([r.name, j, row] )
    prices = pd.DataFrame(prices, columns = ['ride_index','traveller_index','SUBGROUP']).set_index(['ride_index','traveller_index'])
    if 'SUBGROUP' in rm.columns:
        del rm['SUBGROUP']
    rm.index = rm.index.set_names(['ride_index','traveller_index'])

    rm = rm.join(prices)

    rides['total_price_subgroup'] = rm.groupby('ride').sum()['SUBGROUP']  # this is objective fun of matching

    # rm['SUBGROUP'] = rm.apply(lambda x: prices[x.traveller], axis = 1) # this is used for pruning
    rides['SUBGROUP'] = rm.groupby('ride').sum()['SUBGROUP']  # this is objective fun of matching

    rm['desired_{}'.format('SUBGROUP')] = rm.apply(lambda r: rm[rm.traveller == r.traveller].SUBGROUP.min(),
                                                   axis=1)

    inData.sblts.rides_multi_index = rm
    inData.sblts.rides = rides

    return inData
subgroup_split(inData)

DotMap(passengers=            pos  status
1    1519889905       0
2    1435362518       0
3    4301397897       0
4    1571087552       0
5    1679761049       0
..          ...     ...
96     44776722       0
97   2374392592       0
98     44840027       0
99   5235337217       0
100   265090310       0

[100 rows x 2 columns], requests=        origin  destination                treq tdep    ttrav  \
46    44813599   4246422181 2021-04-20 14:57:30  NaN 01:08:16   
47  1552651470     44757287 2021-04-20 14:57:30  NaN 00:32:26   
41  5400602513     44846572 2021-04-20 14:57:33  NaN 01:00:45   
34  2116559760   1478084012 2021-04-20 14:57:34  NaN 01:34:08   
50   736761133     44763130 2021-04-20 14:57:40  NaN 00:55:59   
..         ...          ...                 ...  ...      ...   
83  3669180587   1838103225 2021-04-20 15:02:36  NaN 00:10:41   
90    44837714   1435362394 2021-04-20 15:02:39  NaN 01:11:54   
8   1413910858   1620840079 2021-04-20 15:02:43  NaN 00:12:50   
69    4475

In [ ]:
inData.sblts.rides[['total_group_cost','SUBGROUP']]

In [ ]:
df = pd.DataFrame(rm, columns = ['ride','traveller','SUBGROUP']).set_index(['ride','traveller'])
rm.join(df)

In [ ]:
inData.sblts.rides[['total_group_cost','SUBGROUP']]

In [ ]:
inData = prunings.algo_EXMAS(inData, price_column='SUBGROUP')  # apply pruning strategies for a given pricing strategy

In [ ]:
inData = pipeline.single_eval(inData, params,
                     MATCHING_OBJS = ['total_group_cost'],  # this can be more
                     PRUNINGS = ['EXMAS'],  # and this can be more
                     PRICING = 'SUBGROUP',  # this is taken from first level loop
                     minmax = ['min'], EXPERIMENT_NAME = 'debug', store_res = False)  # direction BPoA, WPoAplot_im(inData)

In [ ]:
inData = prunings.algo_HERMETIC(inData, price_column='SUBGROUP')  # apply pruning strategies for a given pricing strategy
rides = inData.sblts.rides

In [ ]:
rides[rides.selected & ~rides.pruned_HERMETIC].indexes

---
(c) Rafał Kucharski, Delft, 2021